<h2>Import Libraries</h2>

In [1]:
import pandas as pd

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

%load_ext autoreload
%autoreload 2

<p><br>
<h2>Load TEST/TRAIN Data</h2>

In [2]:
SEED = 42

In [3]:
data_train = pd.read_csv('data-train-final.csv', index_col=0)

data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 3607 to 56422
Columns: 1017 entries, installer_cvsmiss to construction_year
dtypes: float64(1017)
memory usage: 369.1 MB


In [4]:
data_train.head()

,installer_cvsmiss,installer_mzeeomari,installer_jaic,installer_government,installer_thomas,installer_mahiti,installer_farm17secondary,installer_laurianrafaeli,installer_scholasticapankrasi,installer_commun,...,waterpoint_type_communalstandpipemultiple,waterpoint_type_other,waterpoint_type_handpump,waterpoint_type_dam,waterpoint_type_improvedspring,gps_height,longitude,latitude,population,construction_year
3607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.760678,0.878073,0.637112,0.300752,0.992548
50870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.022238,0.880156,0.508598,0.000000,0.000000
20413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.022238,0.805545,0.220458,0.000000,0.000000
52806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.022238,0.844227,0.242006,0.000000,0.000000
50091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.383339,0.917896,0.481453,0.225564,0.992052


In [5]:
y_train = pd.read_csv('labels-train-final.csv', index_col=0)

y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47520 entries, 3607 to 56422
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  47520 non-null  object
dtypes: object(1)
memory usage: 742.5+ KB


In [6]:
y_train.head()

,status_group
3607,functional
50870,functional
20413,non functional
52806,non functional
50091,non functional


In [7]:
classes = y_train.status_group.unique()

In [8]:
data_test = pd.read_csv('data-test-final.csv', index_col=0)

data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11880 entries, 2980 to 41264
Columns: 1017 entries, installer_cvsmiss to construction_year
dtypes: float64(1017)
memory usage: 92.3 MB


In [9]:
data_test.head()

,installer_cvsmiss,installer_mzeeomari,installer_jaic,installer_government,installer_thomas,installer_mahiti,installer_farm17secondary,installer_laurianrafaeli,installer_scholasticapankrasi,installer_commun,...,waterpoint_type_communalstandpipemultiple,waterpoint_type_other,waterpoint_type_handpump,waterpoint_type_dam,waterpoint_type_improvedspring,gps_height,longitude,latitude,population,construction_year
2980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.022238,0.792800,0.691285,0.000000,0.000000
5246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.022238,0.813797,0.575521,0.000000,0.000000
22659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.613484,0.879616,0.635858,0.278195,0.997516
39888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.022238,0.821432,0.222333,0.000000,0.000000
13361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.413696,0.848108,0.619679,0.441729,0.999006


In [10]:
y_test = pd.read_csv('labels-test-final.csv', index_col=0)

y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11880 entries, 2980 to 41264
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status_group  11880 non-null  object
dtypes: object(1)
memory usage: 185.6+ KB


In [11]:
y_test.head()

,status_group
2980,non functional
5246,functional
22659,functional
39888,non functional
13361,functional


<p><br>
<h2>Models</h2>

<h3>Random Forest Classifier</h3>
<h4>Build/Train</h4>

In [12]:
# # GridSearchCV hyperparameters TUNING
# param_grid = {
#     'criterion': ['entropy', 'gini'],
#     'bootstrap': [True, False],
#     'max_features': ['auto', 'sqrt'],
#     'max_depth': [50, 100, None],
#     'n_estimators': [100, 500, 1000, 5000, 10000], # 100 is default
#     'class_weight': ['balanced', 'balanced_subsample', None]
#     'min_samples_leaf': [1, 2, 5, 10]
#     'n_jobs': [-1],
#     'random_state': [SEED],
#     'verbose': [1]
# }

# K = 5

# clf = RandomForestClassifier()

# # print(clf.get_params().keys())

# grid_clf = GridSearchCV(
#     clf, 
#     param_grid, 
#     cv=K, 
#     n_jobs=-1, 
#     verbose=20
# )
# grid_clf.fit(data_train, y_train)

# best_parameters = grid_clf.best_params_


# Grid Search found the following optimal parameters: 
# bootstrap: True
# criterion: 'gini'
# max_depth: 50
# max_features: 'auto'
# n_estimators: 1000
# # according to the prior GridSearch...
# best_parameters = {}
# best_parameters['criterion'] = 'gini'
# best_parameters['bootstrap'] = True
# best_parameters['max_depth'] = 50
# best_parameters['max_features'] = 'auto'
# best_parameters['n_estimators'] = 1000



# finished 5/26/2020 @ 10:30 AM PST
# Grid Search found the following optimal parameters: 
# 	bootstrap: True
# 	criterion: 'entropy'
# 	max_depth: 50
# 	max_features: 'auto'
# 	n_estimators: 1000
# 	n_jobs: -1
# 	random_state: 42
# 	verbose: 1

best_parameters = {}
best_parameters['bootstrap'] = True
best_parameters['criterion'] = 'gini'
best_parameters['max_depth'] = 50
best_parameters['max_features'] = 'auto'
best_parameters['n_estimators'] = 2000
best_parameters['class_weight'] = None
best_parameters['min_samples_leaf'] = 1



print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))# # GridSearchCV hyperparameters TUNING

Grid Search found the following optimal parameters: 
	bootstrap: True
	class_weight: None
	criterion: 'gini'
	max_depth: 50
	max_features: 'auto'
	min_samples_leaf: 1
	n_estimators: 2000


In [13]:
clf = RandomForestClassifier(
    oob_score = True,

    bootstrap=best_parameters['bootstrap'],
    criterion=best_parameters['criterion'],
    max_depth=best_parameters['max_depth'],
    max_features=best_parameters['max_features'],
    n_estimators=best_parameters['n_estimators'],

    class_weight=best_parameters['class_weight'],

    min_samples_leaf=best_parameters['min_samples_leaf'], 
    #     # 1: 0.99375,               0.8017676767676768
    #     # 2: 0.8430345117845118,    0.7627104377104377
    #     # 3: 0.781439393939394,     0.7323232323232324 
    #     # 4: 0.75681818181818189,   0.7214646464646465 <-- closest together
    #     # 5: 0.7405723905723905,    0.7095959595959596

    random_state=SEED, 
    n_jobs=-1,
    verbose=1
)

clf.fit(data_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  5.5min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=1,
                       warm_start=False)

<h4>Predict</h4>
<h5>Training Data</h5>

In [14]:
pred_train = clf.predict(data_train)
print("\nscore: ", clf.score(data_train, y_train), "\n")
print("\noob score: ", clf.oob_score_, "\n")
print(classification_report(y_train, pred_train, target_names=classes))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    4.0s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    6.2s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:   10.0s
[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:   11.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    3.6s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    6.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    9.2s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:   13.3s
[Parallel(n_jobs=8)]: Done 2000 

<h5>Testing Data</h5>

In [15]:
pred_test = clf.predict(data_test)
print("\nscore: ", clf.score(data_test, y_test), "\n")
print(classification_report(y_test, pred_test, target_names=classes))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:    3.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    2.1s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done 2000 

In [16]:
# placeholder